In [1]:
from pathlib import Path
from glob import glob
from itertools import cycle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from natsort import index_natsorted
from brokenaxes import brokenaxes

from ase import Atoms, Atom
from pmutt.io import gaussian
from pmutt.statmech import StatMech, trans, vib, rot, elec
from pmutt.constants import kb, Na


# interactive plots
%matplotlib widget

In [2]:
KB_KCAL: float = kb("kcal/K")
AVOGADRO: float = Na

KB_KCAL_MOL: float = KB_KCAL * AVOGADRO
KB_KCAL_MOL


0.001985875020917578

## Path father of the inputs

In [3]:
working_directory = Path.cwd() / "../structures"

if not working_directory.exists():
    raise ValueError("The working directory does not exist.")


## Inputs files
* ADF.2013 outputs
* .txt with 2 columns: 1) name of the file of the output adf and 2) ADF energies in Ev
* G16 outputs

In [4]:
list_file_adf = glob(str(working_directory) + "/sozora/*.out")
list_file_g16 = glob(str(working_directory) + "/g16/sddall/*.log")
energies_file_adf = working_directory / ("sozora/energies.txt")

if not energies_file_adf.exists():
    raise ValueError("The energies file does not exist.")

if not list_file_adf:
    raise ValueError("The list of ADF files does not exist.")

if not list_file_g16:
    raise ValueError("The list of G16 files does not exist.")


## Reading coordinates and frequencies from ADF.2013

In [5]:
find_lines_adf = {}
for file_adf in list_file_adf:
    read_file_adf = Path(file_adf).read_text()

    """ Lines to read the geometry and frequencies form ADF.2013 output """
    start_line_geom = None
    last_line_geom = None
    start_line_freq = None
    last_line_freq = None
    for count, line in enumerate(read_file_adf.split("\n")):

        if (
            line
            == "                                            *  F R E Q U E N C I E S  *"
        ):
            start_line_geom = count + 11
        elif line == " Atomic Masses:":
            last_line_geom = count - 3
        elif line == " List of All Frequencies:":
            start_line_freq = count + 9
        elif line == " Statistical Thermal Analysis  ***  ideal gas assumed  ***":
            last_line_freq = count - 4
        elif start_line_geom and last_line_geom and start_line_freq and last_line_freq:
            break

    if not start_line_geom and not last_line_geom and not start_line_freq and not last_line_freq:
        raise ValueError(f"Geometries or frequencies in {file_adf} of ADF.2013 are not.")

    find_lines_adf[Path(file_adf).stem] = [
        start_line_geom,
        last_line_geom,
        start_line_freq,
        last_line_freq,
    ]

if not find_lines_adf:
    raise ValueError("Geometries and frequencies in ADF.2013 does not exist.")


In [56]:
geoms_systems = {}
freqs_systems = {}
energies_systems = {}
for file_adf in list_file_adf:
    with open(Path(file_adf), "r") as f_adf:
        file_content = f_adf.readlines()

        """Atoms object with ase.Atoms"""
        last_line_geom = find_lines_adf[Path(file_adf).stem][1]
        start_line_geom = find_lines_adf[Path(file_adf).stem][0]
        lines_geom = last_line_geom - start_line_geom
        coord_system = []
        for i in range(lines_geom):
            line = file_content[start_line_geom + i].split()
            coord_system.append(Atom(line[1], (line[5], line[6], line[7])))
        geoms_systems[Path(file_adf).stem] = Atoms(coord_system)

        """Frequencies"""
        last_line_freq = find_lines_adf[Path(file_adf).stem][3]
        start_line_freq = find_lines_adf[Path(file_adf).stem][2]
        lines_freq = last_line_freq - start_line_freq
        freq_system = []
        for i in range(lines_freq):
            line = file_content[start_line_freq + i].split()
            if float(line[0]) > 20.0:
                freq_system.append(float(line[0]))
        freqs_systems[Path(file_adf).stem] = freq_system

    """Energies from ADF.2013 in Ev"""
    with open(energies_file_adf, "r") as f_energies:
        stop = False
        while stop == False:
            line = f_energies.readline().split()
            if line[0] == Path(file_adf).stem:
                energies_systems[Path(file_adf).stem] = float(line[1])
                stop = True

if not geoms_systems:
    raise ValueError("The geometries of the systems does not exist.")
if not freqs_systems:
    raise ValueError("The frequencies of the systems does not exist.")
if not energies_systems:
    raise ValueError("The energies of the systems does not exist.")

geoms_systems["w1s1"].__dict__


{'_cellobj': Cell([0.0, 0.0, 0.0]),
 '_pbc': array([False, False, False]),
 'arrays': {'numbers': array([80,  8,  1,  1]),
  'positions': array([[-0.244886,  0.095186,  0.      ],
         [ 1.858384, -0.18469 ,  0.      ],
         [ 2.317319,  0.18235 ,  0.80465 ],
         [ 2.317319,  0.18235 , -0.80465 ]])},
 '_celldisp': array([[0.],
        [0.],
        [0.]]),
 '_constraints': [],
 'info': {},
 '_calc': None}

## Reading coordinates and frequencies from G16

In [57]:
find_lines_g16 = {}
for file_g16 in list_file_g16:
    read_file_g16 = Path(file_g16).read_text()

    """ Lines to read the geometry form G16 output """
    start_line_geom_g16 = None
    last_line_geom_g16 = None
    for count, line in enumerate(read_file_g16.split("\n")):

        if (
            line
            == "                          Input orientation:                          "
        ):
            start_line_geom_g16 = count + 5
        elif line == "                    Distance matrix (angstroms):":
            last_line_geom_g16 = count - 1
        elif start_line_geom_g16 and last_line_geom_g16:
            break
    find_lines_g16[Path(file_g16).stem] = [start_line_geom_g16, last_line_geom_g16]

if not find_lines_g16:
    raise ValueError("Geometries in G16 does not exist.")


In [58]:
geoms_systems_g16 = {}
freqs_systems_g16 = {}
energies_systems_g16 = {}
for file_g16 in list_file_g16:
    with open(Path(file_g16), "r") as f_g16:
        file_content = f_g16.readlines()

        """Atoms object with ase.Atoms"""
        last_line_geom_g16 = find_lines_g16[Path(file_g16).stem][1]
        start_line_geom_g16 = find_lines_g16[Path(file_g16).stem][0]
        lines_geom_g16 = last_line_geom_g16 - start_line_geom_g16
        coord_system_g16 = []
        for i in range(lines_geom_g16):
            line = file_content[start_line_geom_g16 + i].split()
            coord_system_g16.append(Atom(int(line[1]), (line[3], line[4], line[5])))
        geoms_systems_g16[Path(file_g16).stem] = Atoms(coord_system_g16)

    """Frequencies from G16"""
    freqs_systems_g16[Path(file_g16).stem] = gaussian.read_frequencies(
        file_g16, units="1/cm"
    )
    """Energies from G16 in Ev"""
    energies_systems_g16[Path(file_g16).stem] = gaussian.read_electronic_and_zpe(
        file_g16, units="eV/molecule"
    )

if not geoms_systems_g16:
    raise ValueError("The geometries of the systems does not exist.")
if not freqs_systems_g16:
    raise ValueError("The frequencies of the systems does not exist.")
if not energies_systems_g16:
    raise ValueError("The energies of the systems does not exist.")

geoms_systems_g16["w1s1"].__dict__


{'_cellobj': Cell([0.0, 0.0, 0.0]),
 '_pbc': array([False, False, False]),
 'arrays': {'numbers': array([80,  8,  1,  1]),
  'positions': array([[-0.22513 ,  0.010739,  0.      ],
         [ 1.821376, -0.090165,  0.      ],
         [ 2.325945,  0.18731 ,  0.79892 ],
         [ 2.325945,  0.17731 , -0.79892 ]])},
 '_celldisp': array([[0.],
        [0.],
        [0.]]),
 '_constraints': [],
 'info': {},
 '_calc': None}

In [59]:
print(f"w1s1 Freqs:", freqs_systems_g16["w1s1"])
print(f"and Energy", energies_systems_g16["w1s1"])

w1s1 Freqs: [540.6193, 672.7271, 783.8093, 1725.6588, 3563.1071, 3667.4565]
and Energy -4594.372441886165


In [60]:
"""Translational"""
translation_w1s1 = trans.FreeTrans(
    n_degrees=3, atoms=geoms_systems_g16["w1s1"]
)
"""Vibrational"""
vibration_w1s1 = vib.HarmonicVib(
    vib_wavenumbers=[540.6193, 672.7271, 783.8093, 1725.6588, 3563.1071, 3667.4565]
)  # cm^-1
vibration_w1s1_a = vib.HarmonicVib(
    vib_wavenumbers=[540.6193, 672.7271, 783.8093, 1725.6588, 3563.1071, 3667.4565]
)  # cm^-1
"""Rotational"""
rotation_w1s1 = rot.RigidRotor(
    symmetrynumber=1, atoms=geoms_systems_g16["w1s1"]
)  # simmetry point C1
"""Electronic"""
electronic_w1s1 = elec.GroundStateElec(
    potentialenergy=-4594.372441886165, spin=0
)  # Ev
electronic_w1s1_a = elec.GroundStateElec(
    potentialenergy=-4594.01387237307921511430, spin=0
)  # Ev
"""StatMech Initialization G16"""
statmech_w1s1 = StatMech(
    name="w1s1",
    trans_model=translation_w1s1,
    vib_model=vibration_w1s1,
    rot_model=rotation_w1s1,
    elec_model=electronic_w1s1,
)
"""Thermochemistry"""
H = statmech_w1s1.get_H(T=298.15, units="kcal/mol")
S = statmech_w1s1.get_S(T=298.15, units="kcal/mol/K")
G = statmech_w1s1.get_G(T=298.15, units="kcal/mol")
U = statmech_w1s1.get_U(T=298.15, units="kcal/mol")
print(f"U {U} S {S} H {H} G {G}")
"""StatMech Initialization G16 a"""
statmech_w1s1_a = StatMech(
    name="w1s1",
    trans_model=translation_w1s1,
    vib_model=vibration_w1s1_a,
    rot_model=rotation_w1s1,
    elec_model=electronic_w1s1_a,
)
"""Thermochemistry"""
H_a = statmech_w1s1_a.get_H(T=298.15, units="kcal/mol")
S_a = statmech_w1s1_a.get_S(T=298.15, units="kcal/mol/K")
G_a = statmech_w1s1_a.get_G(T=298.15, units="kcal/mol")
U_a = statmech_w1s1_a.get_U(T=298.15, units="kcal/mol")
print(f"U_a {U_a} S_a {S_a} H_a {H_a} G_a {G_a}")


U -105931.05653417727 S 0.06372378252942931 H -105930.46404942394 G -105949.46329518508
U_a -105922.78772466844 S_a 0.06372378252942931 H_a -105922.19523991509 G_a -105941.19448567624


## Build object to statical mechanical calculation

In [61]:
translation = {}
vibration = {}
rotation = {}
electronic = {}
translation_g16 = {}
vibration_g16 = {}
rotation_g16 = {}
electronic_g16 = {}
for count, file_adf in enumerate(list_file_adf):
    name_file = Path(file_adf).stem

    """Translational"""
    translation[name_file] = trans.FreeTrans(
        n_degrees=3, atoms=geoms_systems[name_file]
    )
    translation_g16[name_file] = trans.FreeTrans(
        n_degrees=3, atoms=geoms_systems_g16[name_file]
    )

    """Vibrational"""
    vibration[name_file] = vib.HarmonicVib(
        vib_wavenumbers=freqs_systems[name_file]
    )  # cm^-1
    vibration_g16[name_file] = vib.HarmonicVib(
        vib_wavenumbers=freqs_systems_g16[name_file]
    )  # cm^-1

    """Rotational"""
    rotation[name_file] = rot.RigidRotor(
        symmetrynumber=1, atoms=geoms_systems[name_file]
    )  # simmetry point C1
    rotation_g16[name_file] = rot.RigidRotor(
        symmetrynumber=1, atoms=geoms_systems_g16[name_file]
    )  # simmetry point C1

    """Electronic"""
    electronic[name_file] = elec.GroundStateElec(
        potentialenergy=float(energies_systems[name_file]), spin=0
    )  # Ev
    electronic_g16[name_file] = elec.GroundStateElec(
        potentialenergy=float(energies_systems_g16[name_file]), spin=0
    )  # Ev

if not translation and not translation_g16:
    raise ValueError("The translational or _g16 part of the systems does not exist.")
if not vibration and not vibration_g16:
    raise ValueError("The vibrational or _g16 part of the systems does not exist.")
if not rotation and not rotation_g16:
    raise ValueError("The rotational or _g16 part of the systems does not exist.")
if not electronic and not electronic_g16:
    raise ValueError("The electronic or _g16 part of the systems does not exist.")


## Calculate the partition function

In [62]:
statmech = {}
statmech_g16 = {}

for file_adf in list_file_adf:
    name_file = Path(file_adf).stem
    """StatMech Initialization ADF.2013"""
    statmech[name_file] = StatMech(
        name=name_file,
        trans_model=translation[name_file],
        vib_model=vibration[name_file],
        rot_model=rotation[name_file],
        elec_model=electronic[name_file],
    )

    """StatMech Initialization G16"""
    statmech_g16[name_file] = StatMech(
        name=name_file,
        trans_model=translation_g16[name_file],
        vib_model=vibration_g16[name_file],
        rot_model=rotation_g16[name_file],
        elec_model=electronic_g16[name_file],
    )

if not statmech:
    raise ValueError("The statmech does not exist.")
if not statmech_g16:
    raise ValueError("The statmech_g16 does not exist.")


## Thermochemistry quantities

In [63]:
values_T = {
    "systems": [],
    "H[kcal/mol]": [],
    "S[kcal/mol/K]": [],
    "G[kcal/mol]": [],
    "U[kcal/mol]": [],
}
for file_adf in list_file_adf:
    name_file = Path(file_adf).stem
    H_statmech = statmech[name_file].get_H(T=298.15, units="kcal/mol")
    S_statmech = statmech[name_file].get_S(T=298.15, units="kcal/mol/K")
    G_statmech = statmech[name_file].get_G(T=298.15, units="kcal/mol")
    U_statmech = statmech[name_file].get_U(T=298.15, units="kcal/mol")
    values_T["systems"].append(name_file)
    values_T["U[kcal/mol]"].append(U_statmech)
    values_T["S[kcal/mol/K]"].append(S_statmech)
    values_T["H[kcal/mol]"].append(H_statmech)
    values_T["G[kcal/mol]"].append(G_statmech)

pd.DataFrame(values_T).sort_values(by="systems",ascending=True,
    key=lambda x: np.argsort(index_natsorted(values_T["systems"])),
    ).reset_index(drop=True)


,systems,H[kcal/mol],S[kcal/mol/K],G[kcal/mol],U[kcal/mol]
0,w1s1,-25267.301616,0.064291,-25286.469943,-25267.894101
1,w2s1,-25668.144242,0.077790,-25691.337472,-25668.736727
2,w3s1,-26028.612760,0.093566,-26056.509399,-26029.205245
3,w3s2,-26016.920429,0.098816,-26046.382560,-26017.512914
4,w3s3,-25993.346695,0.094863,-26021.630140,-25993.939180
5,w4s1,-26380.722574,0.101106,-26410.867402,-26381.315059
6,w4s2,-26376.837751,0.112162,-26410.278856,-26377.430236
7,w4s3,-26370.753345,0.108095,-26402.981940,-26371.345829
8,w4s4,-26361.038428,0.120221,-26396.882235,-26361.630913
9,w5s1,-26722.964409,0.116447,-26757.683222,-26723.556893


In [48]:
values_T_g16 = {
    "systems": [],
    "H[kcal/mol]": [],
    "S[kcal/mol/K]": [],
    "G[kcal/mol]": [],
    "U[kcal/mol]": [],
}
for file_g16 in list_file_g16:
    name_file = Path(file_g16).stem
    H_statmech_g16 = statmech_g16[name_file].get_H(T=298.15, units="kcal/mol")
    S_statmech_g16 = statmech_g16[name_file].get_S(T=298.15, units="kcal/mol/K")
    G_statmech_g16 = statmech_g16[name_file].get_G(T=298.15, units="kcal/mol")
    U_statmech_g16 = statmech_g16[name_file].get_U(T=298.15, units="kcal/mol")
    values_T_g16["systems"].append(name_file)
    values_T_g16["U[kcal/mol]"].append(U_statmech_g16)
    values_T_g16["S[kcal/mol/K]"].append(S_statmech_g16)
    values_T_g16["H[kcal/mol]"].append(H_statmech_g16)
    values_T_g16["G[kcal/mol]"].append(G_statmech_g16)

pd.DataFrame(values_T_g16).sort_values(by="systems",ascending=True,
    key=lambda x: np.argsort(index_natsorted(values_T_g16["systems"])),
    ).reset_index(drop=True)


,systems,H[kcal/mol],S[kcal/mol/K],G[kcal/mol],U[kcal/mol]
0,w1s1,-105930.464049,0.063724,-105949.463295,-105931.056534
1,w2s1,-116669.119843,0.075222,-116691.547298,-116669.712327
2,w3s1,-127373.371340,0.082859,-127398.075669,-127373.963825
3,w3s2,-127370.715323,0.082924,-127395.439046,-127371.307808
4,w3s3,-127339.479391,0.089782,-127366.247955,-127340.071876
5,w4s1,-138073.654465,0.091272,-138100.867244,-138074.246949
6,w4s2,-138066.951458,0.091933,-138094.361333,-138067.543942
7,w4s3,-138064.759815,0.096065,-138093.401723,-138065.352300
8,w4s4,-138068.305951,0.088460,-138094.680316,-138068.898436
9,w5s1,-148759.739722,0.096627,-148788.548935,-148760.332207


## Thermochemistry quantities: different temperatures in kelvin
* ADF.2013
* G16

In [49]:
initital_temperature = 1
final_temperature = 1000
step_temperature = 1


In [50]:

absolute_thermochemistry_per_temperature = {}
absolute_thermochemistry_per_temperature_g16 = {}

for temperature in range(initital_temperature, final_temperature, step_temperature):

    thermo_data = {
        "system": [],
        "H[kcal/mol]": [],
        "S[kcal/mol/K]": [],
        "G[kcal/mol]": [],
        "U[kcal/mol]": [],
    }
    thermo_data_g16 = {
        "system": [],
        "H[kcal/mol]": [],
        "S[kcal/mol/K]": [],
        "G[kcal/mol]": [],
        "U[kcal/mol]": [],
    }

    for file_adf in list_file_adf:
        name_file = Path(file_adf).stem

        """ ADF.2013 """
        H_statmech = statmech[name_file].get_H(T=temperature, units="kcal/mol")
        S_statmech = statmech[name_file].get_S(T=temperature, units="kcal/mol/K")
        G_statmech = statmech[name_file].get_G(T=temperature, units="kcal/mol")
        U_statmech = statmech[name_file].get_U(T=temperature, units="kcal/mol")
        thermo_data["system"].append(name_file)
        thermo_data["H[kcal/mol]"].append(H_statmech)
        thermo_data["S[kcal/mol/K]"].append(S_statmech)
        thermo_data["G[kcal/mol]"].append(G_statmech)
        thermo_data["U[kcal/mol]"].append(U_statmech)
        """ G16 """
        H_statmech_g16 = statmech_g16[name_file].get_H(T=temperature, units="kcal/mol")
        S_statmech_g16 = statmech_g16[name_file].get_S(
            T=temperature, units="kcal/mol/K"
        )
        G_statmech_g16 = statmech_g16[name_file].get_G(T=temperature, units="kcal/mol")
        U_statmech_g16 = statmech_g16[name_file].get_U(T=temperature, units="kcal/mol")
        thermo_data_g16["system"].append(name_file)
        thermo_data_g16["H[kcal/mol]"].append(H_statmech_g16)
        thermo_data_g16["S[kcal/mol/K]"].append(S_statmech_g16)
        thermo_data_g16["G[kcal/mol]"].append(G_statmech_g16)
        thermo_data_g16["U[kcal/mol]"].append(U_statmech_g16)

    absolute_thermochemistry_per_temperature[temperature] = (
        pd.DataFrame(thermo_data)
        .sort_values(
            by="system",
            ascending=True,
            key=lambda x: np.argsort(index_natsorted(thermo_data["system"])),
        )
        .reset_index(drop=True)
    )
    absolute_thermochemistry_per_temperature_g16[temperature] = (
        pd.DataFrame(thermo_data_g16)
        .sort_values(
            by="system",
            ascending=True,
            key=lambda x: np.argsort(index_natsorted(thermo_data_g16["system"])),
            )
        .reset_index(drop=True)
    )


if not absolute_thermochemistry_per_temperature and not absolute_thermochemistry_per_temperature_g16:
    raise ValueError(
        "The absolute thermochemistry or _g16 values of the systems does not exist."
    )


KeyboardInterrupt: 

In [ ]:
absolute_thermochemistry_per_temperature_g16[300].tail()


In [ ]:
absolute_thermochemistry_per_temperature[298].tail()


## DH

In [ ]:
max_number_of_waters = 6

dict_relative_population_delta_H = {
    "system": absolute_thermochemistry_per_temperature[initital_temperature]["system"]
}

for temperature in range(initital_temperature, final_temperature, step_temperature):

    list_of_delta_H = list()

    for water in range(1, max_number_of_waters + 1):

        reference_system = f"w{water}s1"

        H_reference = float(
            absolute_thermochemistry_per_temperature[temperature][
                absolute_thermochemistry_per_temperature[temperature]["system"]
                == reference_system
            ]["H[kcal/mol]"]
        )

        H_values = absolute_thermochemistry_per_temperature[temperature][
            absolute_thermochemistry_per_temperature[temperature][
                "system"
            ].str.startswith(reference_system[:-1])
        ]["H[kcal/mol]"]

        delta_H = H_values - H_reference

        total_delta_H = np.exp(-delta_H / KB_KCAL_MOL / temperature).sum()

        population_delta_H = (
            np.exp(-delta_H / KB_KCAL_MOL / temperature) / total_delta_H
        )

        list_of_delta_H.append(population_delta_H)

    dict_relative_population_delta_H[f"T={temperature}"] = [
        value for sublist in np.array(list_of_delta_H).T for value in sublist
    ]


df_relative_population_delta_H = pd.DataFrame(dict_relative_population_delta_H)


## DG

In [ ]:
max_number_of_waters = 6

dict_relative_population_delta_G = {
    "system": absolute_thermochemistry_per_temperature[initital_temperature]["system"]
}

for temperature in range(initital_temperature, final_temperature, step_temperature):

    list_of_delta_G = list()

    for water in range(1, max_number_of_waters + 1):

        reference_system = f"w{water}s1"

        G_reference = float(
            absolute_thermochemistry_per_temperature[temperature][
                absolute_thermochemistry_per_temperature[temperature]["system"]
                == reference_system
            ]["G[kcal/mol]"]
        )

        G_values = absolute_thermochemistry_per_temperature[temperature][
            absolute_thermochemistry_per_temperature[temperature][
                "system"
            ].str.startswith(reference_system[:-1])
        ]["G[kcal/mol]"]

        delta_G = G_values - G_reference

        total_delta_G = np.exp(-delta_G / KB_KCAL_MOL / temperature).sum()

        population_delta_G = (
            np.exp(-delta_G / KB_KCAL_MOL / temperature) / total_delta_G
        )

        list_of_delta_G.append(population_delta_G)

    dict_relative_population_delta_G[f"T={temperature}"] = [
        value for sublist in np.array(list_of_delta_G).T for value in sublist
    ]


df_relative_population_delta_G = pd.DataFrame(dict_relative_population_delta_G)


In [ ]:
df_relative_population_delta_G[["system", "T=300"]].tail()


## DU

In [ ]:
max_number_of_waters = 6

dict_relative_population_delta_U = {
    "system": absolute_thermochemistry_per_temperature[initital_temperature]["system"]
}

for temperature in range(initital_temperature, final_temperature, step_temperature):

    list_of_delta_U = list()

    for water in range(1, max_number_of_waters + 1):

        reference_system = f"w{water}s1"

        U_reference = float(
            absolute_thermochemistry_per_temperature[temperature][
                absolute_thermochemistry_per_temperature[temperature]["system"]
                == reference_system
            ]["U[kcal/mol]"]
        )

        U_values = absolute_thermochemistry_per_temperature[temperature][
            absolute_thermochemistry_per_temperature[temperature][
                "system"
            ].str.startswith(reference_system[:-1])
        ]["U[kcal/mol]"]

        delta_U = U_values - U_reference

        total_delta_U = np.exp(-delta_U / KB_KCAL_MOL / temperature).sum()

        population_delta_U = (
            np.exp(-delta_U / KB_KCAL_MOL / temperature) / total_delta_U
        )

        list_of_delta_U.append(population_delta_U)

    dict_relative_population_delta_U[f"T={temperature}"] = [
        value for sublist in np.array(list_of_delta_U).T for value in sublist
    ]


df_relative_population_delta_U = pd.DataFrame(dict_relative_population_delta_U)


## DS

In [ ]:
max_number_of_waters = 6

dict_relative_population_delta_S = {
    "system": absolute_thermochemistry_per_temperature[initital_temperature]["system"]
}

for temperature in range(initital_temperature, final_temperature, step_temperature):

    list_of_delta_S = list()

    for water in range(1, max_number_of_waters + 1):

        reference_system = f"w{water}s1"

        S_reference = float(
            absolute_thermochemistry_per_temperature[temperature][
                absolute_thermochemistry_per_temperature[temperature]["system"]
                == reference_system
            ]["S[kcal/mol/K]"]
        )

        S_values = absolute_thermochemistry_per_temperature[temperature][
            absolute_thermochemistry_per_temperature[temperature][
                "system"
            ].str.startswith(reference_system[:-1])
        ]["S[kcal/mol/K]"]

        delta_S = S_values - S_reference

        total_delta_S = np.exp(-delta_S / KB_KCAL_MOL / temperature).sum()

        population_delta_S = (
            np.exp(-delta_S / KB_KCAL_MOL / temperature) / total_delta_S
        )

        list_of_delta_S.append(population_delta_S)

    dict_relative_population_delta_S[f"T={temperature}"] = [
        value for sublist in np.array(list_of_delta_S).T for value in sublist
    ]


df_relative_population_delta_S = pd.DataFrame(dict_relative_population_delta_S)


In [ ]:
# - lines style in a for loop
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d', "-.","--",":"]
# lines = ['s', '^', 'v', '<', '>', 'o', 's', 'd', '.'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-+', '-x', '-d', '-1', '-2', '-3', '-4', '-h', '-p', '-|', '-_', '-D', '-H']
lines = ["-", "--", "-.", ":"]

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]




linecycler = cycle(lines)
cm = cycle(color_palette)

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.grid()

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w6s"):
        continue

    ax.plot(
        temp,
        df_relative_population_delta_G[
            df_relative_population_delta_G["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


ax.xaxis.set_major_locator(MultipleLocator(100))

ax.legend(loc="upper right", fontsize=8, ncol=5)
ax.set_xlabel(r"Temperature K", fontsize=12)
ax.set_ylabel(r"Relative Population $\Delta$G", fontsize=12)


In [ ]:
# - lines style in a for loop
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d', "-.","--",":"]
# lines = ['s', '^', 'v', '<', '>', 'o', 's', 'd', '.'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-+', '-x', '-d', '-1', '-2', '-3', '-4', '-h', '-p', '-|', '-_', '-D', '-H']
lines = ["-", "--", "-.", ":"]

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

linecycler = cycle(lines)
cm = cycle(color_palette)

fig = plt.figure(figsize=(10, 6))

grid = plt.GridSpec(2, 2, wspace=0.4, hspace=0.3)

ax1 = plt.subplot(grid[0, 0])
ax1.grid()

ax2 = plt.subplot(grid[0, 1])
ax2.grid()

ax3 = plt.subplot(grid[1, 0])
ax3.grid()

ax4 = plt.subplot(grid[1, 1])
ax4.grid()

# ------------------------------------------------------------------------------
temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w6s"):
        continue

    ax1.plot(
        temp,
        df_relative_population_delta_G[
            df_relative_population_delta_G["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


ax1.xaxis.set_major_locator(MultipleLocator(100))

# ax1.legend(loc="upper right", fontsize=8, ncol=5)
# ax1.set_xlabel(r"Temperature K", fontsize=12)
ax1.set_ylabel(r"Relative Population $\Delta$G", fontsize=12)
ax1.xaxis.set_tick_params(labelsize=8)

# ------------------------------------------------------------------------------

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w6s"):
        continue

    ax2.plot(
        temp,
        df_relative_population_delta_H[
            df_relative_population_delta_H["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


ax2.xaxis.set_major_locator(MultipleLocator(100))

# ax2.legend(loc="upper right", fontsize=8, ncol=5)
# ax2.set_xlabel(r"Temperature K", fontsize=12)
ax2.xaxis.set_tick_params(labelsize=8)
ax2.set_ylabel(r"Relative Population $\Delta$H", fontsize=12)
ax2.xaxis.set_tick_params(labelsize=8)

# ------------------------------------------------------------------------------

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w6s"):
        continue

    ax3.plot(
        temp,
        df_relative_population_delta_S[
            df_relative_population_delta_S["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


ax3.xaxis.set_major_locator(MultipleLocator(100))

# ax3.legend(loc="upper right", fontsize=8, ncol=5)
# ax3.set_xlabel(r"Temperature K", fontsize=12)
ax3.xaxis.set_tick_params(labelsize=8)
ax3.set_ylabel(r"Relative Population $\Delta$S", fontsize=12)
ax3.xaxis.set_tick_params(labelsize=8)

# ------------------------------------------------------------------------------

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w6s"):
        continue

    ax4.plot(
        temp,
        df_relative_population_delta_U[
            df_relative_population_delta_U["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


ax4.xaxis.set_major_locator(MultipleLocator(100))

# ax4.legend(loc="upper right", fontsize=8, ncol=5)
# ax4.set_xlabel(r"Temperature K", fontsize=12)
ax4.xaxis.set_tick_params(labelsize=8)
ax4.set_ylabel(r"Relative Population $\Delta$U", fontsize=12)
ax4.xaxis.set_tick_params(labelsize=8)

In [ ]:
# - lines style in a for loop
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d', "-.","--",":"]
# lines = ['s', '^', 'v', '<', '>', 'o', 's', 'd', '.'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-+', '-x', '-d', '-1', '-2', '-3', '-4', '-h', '-p', '-|', '-_', '-D', '-H']
lines = ["-", "--", "-.", ":"]

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

linecycler = cycle(lines)
cm = cycle(color_palette)

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.grid()

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w5s"):
        continue

    ax.plot(
        temp,
        df_relative_population_delta_G[
            df_relative_population_delta_G["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )

ax.xaxis.set_major_locator(MultipleLocator(100))
ax.legend(loc="upper right", fontsize=8, ncol=3)
ax.set_xlabel(r"Temperature K", fontsize=12)
ax.set_ylabel(r"Relative Population $\Delta$G", fontsize=12)


In [ ]:
# - lines style in a for loop
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d', "-.","--",":"]
# lines = ['s', '^', 'v', '<', '>', 'o', 's', 'd', '.'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-+', '-x', '-d', '-1', '-2', '-3', '-4', '-h', '-p', '-|', '-_', '-D', '-H']
lines = ["-", "--", "-.", ":"]

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

linecycler = cycle(lines)
cm = cycle(color_palette)

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.grid()

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w4s"):
        continue

    ax.plot(
        temp,
        df_relative_population_delta_G[
            df_relative_population_delta_G["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )

ax.xaxis.set_major_locator(MultipleLocator(100))


ax.legend(loc="center left", fontsize=10, ncol=1)
ax.set_xlabel(r"Temperature K", fontsize=12)
ax.set_ylabel(r"Relative Population $\Delta$G", fontsize=12)


In [ ]:
# - lines style in a for loop
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d', "-.","--",":"]
# lines = ['s', '^', 'v', '<', '>', 'o', 's', 'd', '.'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-+', '-x', '-d', '-1', '-2', '-3', '-4', '-h', '-p', '-|', '-_', '-D', '-H']
lines = ["-", "--", "-.", ":"]

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

linecycler = cycle(lines)
cm = cycle(color_palette)

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.grid()

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w3s"):
        continue

    ax.plot(
        temp,
        df_relative_population_delta_G[
            df_relative_population_delta_G["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


ax.legend(loc="center left", fontsize=10, ncol=3)
ax.xaxis.set_major_locator(MultipleLocator(100))

ax.set_xlabel(r"Temperature K", fontsize=12)
ax.set_ylabel(r"Relative Population $\Delta$G", fontsize=12)


In [ ]:
# - lines style in a for loop
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-d', "-.","--",":"]
# lines = ['s', '^', 'v', '<', '>', 'o', 's', 'd', '.'] #, '-h', '-p', '-D', '-H']
# lines = ['-o', '-^', '-v', '-<', '->', '-s', '-+', '-x', '-d', '-1', '-2', '-3', '-4', '-h', '-p', '-|', '-_', '-D', '-H']
lines = ["-", "--", "-.", ":"]

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

linecycler = cycle(lines)
cm = cycle(color_palette)

fig = plt.figure(figsize=(6, 4))
# ax = fig.add_subplot(111)
# ax.grid()

bax = brokenaxes( ylims=((-0.01, 0.07), (0.9, 1.01)), hspace=.2)
bax.grid()

temp = list(range(initital_temperature, final_temperature, step_temperature))
for system in df_relative_population_delta_G["system"]:

    if not system.startswith("w3s"):
        continue

    bax.plot(
        temp,
        df_relative_population_delta_G[
            df_relative_population_delta_G["system"] == system
        ].T.iloc[1:, 0],
        label=system[2:],
        color=next(cm),
        linestyle=next(linecycler),
    )


bax.legend(loc="center left", fontsize=10, ncol=3)
# bax.xaxis.set_major_locator(MultipleLocator(100))
bax.locator_params(nbins=10, axis='y')
bax.locator_params(nbins=20, axis='x')

bax.set_xlabel(r"Temperature K", fontsize=12)
bax.set_ylabel(r"Relative Population $\Delta$G", fontsize=12)
